Benjamin Ye  
CS/CNE/EE 156a: Learning Systems (Fall 2023)  
November 20, 2023

## Homework 8

In [1]:
import numpy as np
import pandas as pd
import requests
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

DATA_DIR = globals()['_dh'][0].parents[1] / "data"
rng = np.random.default_rng()

In [2]:
DATA_DIR.mkdir(exist_ok=True)
data = {}
for dataset in ["train", "test"]:
    file = f"features.{dataset}"
    if not (DATA_DIR / file).exists():
        r = requests.get(f"http://www.amlbook.com/data/zip/{file}")
        with open(DATA_DIR / file, "wb") as f:
            f.write(r.content)
    data[dataset] = np.loadtxt(DATA_DIR / file)

### Problems 2–4

In [3]:
C = 0.01
Q = 2
clf = svm.SVC(C=C, kernel="poly", degree=Q, gamma=1, coef0=1)
df = pd.DataFrame(columns=["classifier", "number of support vectors", 
                           "in-sample error", "out-of-sample error"])
for digit in range(10):
    x_train = data["train"][:, 1:]
    y_train = 2 * (data["train"][:, 0] == digit) - 1
    clf.fit(x_train, y_train)
    df.loc[digit] = (
        f"{digit} vs. all", 
        clf.n_support_.sum(),
        1 - clf.score(x_train, y_train),
        1 - clf.score(data["test"][:, 1:], 
                      2 * (data["test"][:, 0] == digit) - 1)
    )
(df.style.hide(axis="index")
         .set_caption(f"Soft margin ({C=}) SVM with polynomial kernel ({Q=})"))

classifier,number of support vectors,in-sample error,out-of-sample error
0 vs. all,2179,0.105884,0.111609
1 vs. all,386,0.014401,0.021923
2 vs. all,1970,0.100261,0.098655
3 vs. all,1950,0.090248,0.082711
4 vs. all,1856,0.089425,0.099651
5 vs. all,1585,0.076258,0.079721
6 vs. all,1893,0.091071,0.084704
7 vs. all,1704,0.088465,0.073244
8 vs. all,1776,0.074338,0.082711
9 vs. all,1978,0.088328,0.088191


### Problems 5–6

In [4]:
x_train = data["train"][np.isin(data["train"][:, 0], (1, 5))]
y_train = 2 * (x_train[:, 0] == 1) - 1
x_test = data["test"][np.isin(data["test"][:, 0], (1, 5))]
y_test = 2 * (x_test[:, 0] == 1) - 1
df = pd.DataFrame(columns=["C", "Q", "number of support vectors",
                           "in-sample error", "out-of-sample error"])
for Q in (2, 5):
    for C in (0.0001, 0.001, 0.01, 0.1, 1):
        clf = svm.SVC(C=C, kernel="poly", degree=Q, gamma=1, coef0=1)
        clf.fit(x_train[:, 1:], y_train)
        df.loc[len(df)] = (
            C, Q, clf.n_support_.sum(),
            1 - clf.score(x_train[:, 1:], y_train),
            1 - clf.score(x_test[:, 1:], y_test)
        )
(df.style.hide(axis="index")
         .format({"C": "{:.4f}", "Q": "{:.0f}", "number of support vectors": "{:.0f}"})
         .set_caption(f"Soft margin ({C=}) SVM with polynomial "
                      f"kernel ({Q=}) for 1 vs. 5 classifier"))

C,Q,number of support vectors,in-sample error,out-of-sample error
0.0001,2,236,0.008969,0.016509
0.0010,2,76,0.004484,0.016509
0.0100,2,34,0.004484,0.018868
0.1000,2,24,0.004484,0.018868
1.0000,2,24,0.003203,0.018868
0.0001,5,26,0.004484,0.018868
0.0010,5,25,0.004484,0.021226
0.0100,5,23,0.003844,0.021226
0.1000,5,25,0.003203,0.018868
1.0000,5,21,0.003203,0.021226


### Problems 7–8

In [5]:
Cs = (0.0001, 0.001, 0.01, 0.1, 1)
Q = 2
N_runs = 100
N_folds = 10
clfs = [svm.SVC(C=C, kernel="poly", degree=Q, gamma=1, coef0=1) 
        for C in Cs]
x_train = data["train"][np.isin(data["train"][:, 0], (1, 5))]
y_train = 2 * (x_train[:, 0] == 1) - 1
counters = np.zeros((2, len(Cs)), dtype=float)
for _ in range(N_runs):
    Es_cv = tuple(1 - cross_val_score(clf, x_train[:, 1:], y_train, 
                                      cv=N_folds).mean()
                  for clf in clfs)
    counters[0] += Es_cv
    counters[1, np.argmin(Es_cv)] += 1
    x_train, y_train = shuffle(x_train, y_train)
counters /= N_runs
df = pd.DataFrame({"C": Cs, "cross-validation error": counters[0],
                   "selection rate": counters[1]})
df.style.hide(axis="index").format({"C": "{:.4f}", "selection rate": "{:.2f}"})

C,cross-validation error,selection rate
0.0001,0.009755,0.00
0.0010,0.004721,0.57
0.0100,0.004657,0.20
0.1000,0.004785,0.09
1.0000,0.004907,0.14


### Problems 9–10

In [6]:
x_train = data["train"][np.isin(data["train"][:, 0], (1, 5))]
y_train = 2 * (x_train[:, 0] == 1) - 1
x_test = data["test"][np.isin(data["test"][:, 0], (1, 5))]
y_test = 2 * (x_test[:, 0] == 1) - 1
df = pd.DataFrame(columns=["C", "in-sample error", "out-of-sample error"])
for C in (0.01, 1, 100, 1e4, 1e6):
    clf = svm.SVC(C=C, gamma=1)
    clf.fit(x_train[:, 1:], y_train)
    df.loc[len(df)] = (
        clf.C, 
        1 - clf.score(x_train[:, 1:], y_train),
        1 - clf.score(x_test[:, 1:], y_test)
    )
df.style.hide(axis="index").format({"C": "{:g}"})

C,in-sample error,out-of-sample error
0.01,0.003844,0.023585
1,0.004484,0.021226
100,0.003203,0.018868
10000,0.002562,0.023585
1e+06,0.000641,0.023585
